# TMDB 5000 영화 데이터 세트
- IMDB의 영화 중, 주요 5000개 영화에 대한 메타 정보를 가공해 Kaggle에서 제공하고 있는 데이터셋
- tmdb_5000_credits.csv, tmdb_5000_movies.csv 두 개의 파일을 가지고 실행

### 1. 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- 그저 콘텐츠가 비슷한지 아닌지만 따져서 필터링하는 기초 중의 기초 알고리즘

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
movies_path = '../../_data-examples/tmdb_5000/tmdb_5000_movies.csv'
credit_path = '../../_data-examples/tmdb_5000/tmdb_5000_credits.csv'

In [5]:
movies = pd.read_csv(movies_path)
print(movies.shape) # (4803, 20) : 4803개의 영화에 대해, 20개의 feature 존재
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [6]:
# 컨텐츠 기반 필터링 추천에 사용할 칼럼만 추출해서, 새롭게 df를 만든다.
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
                    'keywords', 'overview']]

In [9]:
movies_df.head(2)
# genres, keywords의 데이터 형태가 리스트 안에 여러 개의 dictionary로 되어 있음. 전처리가 필요!

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."


In [10]:
# ast 모듈의 literal_eval을 이용하면 문자열을 적절한 데이터구조로 바꿔준다.... 알면 좋겠군요
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [13]:
movies_df[['genres', 'keywords']].head(3)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."


In [14]:
# 리스트의 각 딕셔너리에 대해, name만 뽑아와서 리스트로 정리한다. 즉 id를 제거
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


In [15]:
# 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환한다.
# genres를 피처 벡터화 행렬로 변환한 데이터셋을 코사인 유사도를 이용해 비교
# 장르 유사도가 높은 영화 중에, 평점이 높은 순으로 영화를 추천

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해, 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [17]:
# cosine 유사도 확인
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [ ]:
# genre_sim은 결국, 각 영화(4803개) 에 대해, 다른 영화들의 장르 유사도를 가지고 있다.
# 비교 대상이 되는 행의 유사도 값이 높은 순으로, 정렬된 행렬의 위치 인덱스 값 추출(argsort 사용)
